In [1]:
MODEL = "llama2:latest"

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_community.embeddings import OllamaEmbeddings

model = OllamaLLM(model=MODEL)
embedding = OllamaEmbeddings()
# model.invoke('tell me a joke')


C:\Users\sagar\AppData\Local\Temp\ipykernel_12132\3187340122.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings()


In [3]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser
# chain.invoke('what is the capital of India')

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Blackbook-Sagar.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 0, 'page_label': '1'}, page_content='INPLANT TRAINING REPORT \n \n \n \nINSTRUMENTATION ENGINEERING DEPARTMENT \n \n \n \n \n  GOVERNMENT POLYTECHNIC, MUMBAI \n \n  \n(An Autonomous Institute of Government of Maharashtra)  \n49, Ali Yavar Jung Marg, Kherwadi, Bandra (East), Mumbai – 400 051  \nWebsite: www.gpmumbai.ac.in \n \n \n \nNAME OF STUDENT:  SAGAR  RAVINDRA  MANCHAKATLA \nPROGRAMME:  INSTRUMENTATION ;   SEMESTER/YEAR:  6th/3th \n     ENROLMENT No.:        FS20IS002   \nCONTACT No.:      9987799356'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_page

In [6]:
from langchain.prompts import PromptTemplate
template = """
    Answer the question based on the context below. If you can't 
    answer the question, reply "I don't know".

    Context: {context}

    Question: {question}
"""
prompt = PromptTemplate.from_template(template=template)

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'Hi there! My name is Santiago, just like my parents named me. Nice to meet you!'

In [ ]:
chain.input_schema.schema()

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding)

c:\OLLAMA\local-RAG\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke('Government Polytechnic')

[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 31, 'page_label': '32'}, page_content='32 | P a g e   \nInstrument Index'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 6, 'page_label': '7'}, page_content='7 | P a g e'),
 Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 14, 'page_label': '15'}, page_content='15 | P a g e   \nPiping Design services: \n\uf0b7 Static and dynamic stress anal

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [10]:
questions = [
    # 'How many internships i have completed',
    # 'How many projects i have completed'
    # 'What is my education'
    'Roles and responsibilities of each personnel'
]

for question in questions:
    print(f"Question : {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question : Roles and responsibilities of each personnel
Answer:  Based on the document provided, the following are the roles and responsibilities of each personnel involved in the project:

1. Admin:
	* Created the documents (Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 19, 'page_label': '18'}, page_content='18 | P a g e   \n\uf0b7 Checked the quantity of various instruments such as Pressure Gauges and Pressure Transmitters in P&ID with respect to its symbols.')
	* Modified the documents (Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-05-31T10:55:10+00:00', 'author': 'Admin', 'moddate': '2023-05-31T10:55:10+00:00', 'source': 'Blackbook-Sagar.pdf', 'total_pages': 46, 'page': 39, 'page_label': '40'}, page_content='40 | P a g e

In [12]:
chain.batch([{"question": q} for q in questions])


['Based on the provided PDF document, you have completed 3 internships:\n\n1. Full Stack Intern at Converse AI Labs from August 2025 to July 2025\n2. MERN and Python Mentor at Static Int. Educare from July 2024\n3. Project-based learning mentor at Endless Runner from January 2023 to April 2023\n\nTherefore, the answer to your question is "3 internships".',
 'Based on the provided PDF document, you have completed 7 projects. Your education is listed as "Thadomal Shahani Engineering College Mumbai, India" where you studied Computer Engineering and earned a Bachelor of Engineering degree with a CGPA of 9.34 from Aug. 2023 to May 2026, followed by a Diploma in Instrumentation Engineering from Government Polytechnic Mumbai, India with a percentage of 94 from Aug. 2020 to May 2023.']

In [ ]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)